In [ ]:
%reset -f

In [ ]:

import pandas as pd
import numpy as np
from pathlib import Path
import ssl
from email.message import EmailMessage
import os
import smtplib

bases_path = Path("bases")
listagem_path = Path("listagem")

relatorio = bases_path / "relatorio_ra-todas_02-12-2025.xlsx"
listagem = listagem_path / "listagem.xlsx"
df_relatorio  = pd.read_excel(relatorio, engine="calamine", header=5, sheet_name='Relatório')
df_lista = pd.read_excel(listagem, engine="calamine")




In [ ]:
df_relatorio.head(2)

In [ ]:
df_relatorio.columns

In [ ]:
df_relatorio['Sem_Estoque'] = (df_relatorio['Ruptura'] + df_relatorio['Faltou no Estoque']).abs()

In [ ]:
df_relatorio['Sem_Estoque'].sum()

In [ ]:
df_relatorio = df_relatorio.drop(columns='Unnamed: 33')

In [ ]:
df_relatorio.dtypes

In [ ]:
df_lista.head()

In [ ]:
df_lista.dtypes

In [ ]:
df_lista.columns

In [ ]:
df_relatorio = df_relatorio.merge(df_lista[['Cod. Loja', 'Franqueado']], how='left', on='Cod. Loja')

In [ ]:
df_relatorio['Franqueado'].head()

In [ ]:
df_relatorio.isnull().sum()

In [ ]:
print("validação 1...")
pedidos_sem_retorno = df_relatorio[df_relatorio['Retorno Pedido'] == '-']
condicao_1 = (pedidos_sem_retorno['Ressuprir'].sum() > 0) or (pedidos_sem_retorno['Atendido (SAP)'] > 0).any()

if condicao_1:
    print("!! ALERTA: Há pedidos sem retorno, com 'Ressuprir' ou 'Atendido maior que 0")
else: 
    print("OK: Não há pedidos sem retorno com 'Ressuprir' ou 'Atendido' maior que 0")

print("-"*50)

print("Validação 2...")

filtro_cancelada = df_relatorio['Cancelada'] == 0
filtro_atendido_sap = df_relatorio['Atendido (SAP)'] == 0
filtro_necessidade = df_relatorio['Necessidade'] > df_relatorio['Sem_Estoque']

pedidos_inconsistentes = df_relatorio[filtro_cancelada & filtro_atendido_sap & filtro_necessidade]

if not pedidos_inconsistentes.empty:
    print("!! ALERTA: Há pedidos com necessidade maior que Sem_Estoque, não foram atendidos e não constam como cancelados")
else:
    print("OK: Todos os pedidos com necessidade maior que o estoque foram tratados corretamente")

    

In [ ]:
def formatar_moeda(valor):
    return f'R$ {valor:,.2f}'.replace(',', 'X').replace('.', ',').replace('X', '.')

def formatar_qtd(valor):
    return f'{valor:,.0f}'.replace(',', '.')

In [ ]:
filtro = df_relatorio['Sem_Estoque'] > 0
df_filtrado = df_relatorio[filtro].copy()

TD_STK = df_filtrado.groupby(['Grife', 'Artigo', 'Cor', 'Tamanho'])[['Sem_Estoque']].sum().reset_index()


soma_total = TD_STK['Sem_Estoque'].sum()

# 3. Criar a linha de Total manualmente
# Como temos várias colunas de texto (Artigo, Cor, etc), precisamos decidir o que colocar nelas (deixei vazio '')
nova_linha = {
    'Grife': 'Total Geral',
    'Artigo': '',
    'Cor': '',
    'Tamanho': '',
    'Sem_Estoque': soma_total
}

# 4. Transformar em DataFrame e Concatenar
df_total_B = pd.DataFrame([nova_linha])
TD_STK = pd.concat([TD_STK, df_total_B], ignore_index=True)

TD_STK['Sem_Estoque'] = TD_STK['Sem_Estoque'].apply(formatar_qtd)
# Visualizar
TD_STK.head() # Mostra apenas o final para confirmar o total

In [ ]:
cols_to_sum = ['Necessidade', 'Sem_Estoque', 'Cancelada', 'Atendido (SAP)', 'Valor']

for col in cols_to_sum:
    # 'coerce' transforma textos estranhos em NaN (vazio), depois preenchemos com 0
    df_relatorio[col] = pd.to_numeric(df_relatorio[col], errors='coerce').fillna(0)

    
TD_RESUMO = df_relatorio.groupby('Grife')[['Necessidade', 'Sem_Estoque', 'Cancelada', 'Atendido (SAP)', 'Valor']].sum()
TD_RESUMO = TD_RESUMO.reset_index()
TD_RESUMO

In [ ]:
cols_numericas = ['Necessidade', 'Sem_Estoque', 'Cancelada', 'Atendido (SAP)', 'Valor']
TD_RESUMO = df_relatorio.groupby('Grife')[cols_numericas].sum().reset_index()

# 2. CALCULA O TOTAL GERAL
# Somamos apenas as colunas numéricas
linha_total = TD_RESUMO[cols_numericas].sum()
# Definimos o nome da primeira coluna para aparecer na linha de baixo
linha_total['Grife'] = 'Total Geral'

# 3. ADICIONA A LINHA DE TOTAL AO DATAFRAME
# Transformamos a série em DataFrame e concatenamos
df_total = pd.DataFrame([linha_total])
TD_RESUMO = pd.concat([TD_RESUMO, df_total], ignore_index=True)


# Aplica formatação de moeda na coluna Valor
TD_RESUMO['Valor'] = TD_RESUMO['Valor'].apply(formatar_moeda)

# Aplica formatação de quantidade nas outras colunas
cols_qtd = ['Necessidade', 'Sem_Estoque', 'Cancelada', 'Atendido (SAP)']
for col in cols_qtd:
    TD_RESUMO[col] = TD_RESUMO[col].apply(formatar_qtd)


display(TD_RESUMO)

In [ ]:
filtro_cancelada = df_relatorio['Cancelada'] > 0
df_cancelado = df_relatorio[filtro_cancelada].copy()
TD_CANC = df_cancelado.groupby('Franqueado')[['Necessidade', 'Cancelada']].sum().reset_index()

total_nec = TD_CANC['Necessidade'].sum()
total_canc = TD_CANC['Cancelada'].sum()

linha_total = {
    'Franqueado': 'Total Geral',
    'Necessidade': total_nec,
    'Cancelada': total_canc
}


TD_CANC = pd.concat([TD_CANC, pd.DataFrame([linha_total])], ignore_index=True)

col_qtd = ['Necessidade', 'Cancelada']

for col in col_qtd:
    TD_CANC[col] = TD_CANC[col].apply(formatar_qtd)


TD_CANC

In [ ]:
cols_colc = ['Necessidade', 'Sem_Estoque', 'Cancelada', 'Atendido (SAP)', 'Valor']

for col in cols_colc:
    df_relatorio[col] = pd.to_numeric(df_relatorio[col], errors='coerce').fillna(0)


tot_necessidade = df_relatorio['Necessidade'].sum()
tot_sem_estoque = df_relatorio['Sem_Estoque'].sum()
tot_cancelado   = df_relatorio['Cancelada'].sum()
tot_atendido    = df_relatorio['Atendido (SAP)'].sum()
tot_receita     = df_relatorio['Valor'].sum()

# Cálculos de Porcentagem
pct_sem_estoque = (tot_sem_estoque / tot_necessidade * 100) if tot_necessidade > 0 else 0
pct_cancelado   = (tot_cancelado / tot_necessidade * 100) if tot_necessidade > 0 else 0


def fmt_br(valor, eh_moeda=False):
    if eh_moeda:
        texto = f'{valor:,.2f}' #Adiciona vírgula como separador de milhar e garante sempre 2 casas decimais.
        return texto.replace(',', 'X').replace('.', ',').replace('X', '.')
    else:
        texto = f'{valor:,.0f}'
        return texto.replace(',', '.')
    

    

In [ ]:
nome_arquivo_final = 'RA.FRQ.xlsx'

with pd.ExcelWriter(nome_arquivo_final, engine='xlsxwriter') as writer: #pd.ExcelWriter: É como comprar uma pasta vazia e deixá-la aberta em cima da mesa. Ela serve para "agrupar" as folhas.
    #writer: É o nome que você deu para essa pasta enquanto ela está aberta na sua mão.
    #df.to_excel(writer...): É a ação de imprimir uma folha e colocá-la dentro dessa pasta específica, em vez de jogar a folha solta na mesa.
    #with: É um segurança automático. Ele garante que, assim que você terminar de colocar a última folha (ou se o prédio pegar fogo no meio do processo), a pasta será fechada, lacrada e guardada no armário corretamente.
    df_relatorio.to_excel(writer, sheet_name='Relatório', index=False)
    TD_STK.to_excel(writer, sheet_name='ESTOQUE', index=False)
    TD_CANC.to_excel(writer, sheet_name='CANCELADO', index=False)
    TD_RESUMO.to_excel(writer, sheet_name='TD_RESUMO', index=False)

print(f"Arquivo {nome_arquivo_final} salvo. Iniciando envio de e-mail...")

# ATENÇÃO: Digite a senha manualmente, sem copiar/colar para evitar caracteres especiais
email_origem = "gabriel.thiago@ciahering.com.br"
senha_app = "sbmvcqubqmtvoxby"  # Digite sua senha de app aqui, sem espaços ou caracteres especiais
email_destino = "giulia.freitas@ciahering.com.br"

msg = EmailMessage()
msg['Subject'] = "R.A 2.0 | Franquia 02.12 (TESTE AUTOMAÇÃO)"
msg['From'] = email_origem
msg['To'] = email_destino

corpo_html =f"""
<html>
    <body>
        <p>Olá, equipe!</p>
        <p>Segue abaixo o resumo das principais informações referentes ao abastecimento de R.A 2.0 desta semana (Franquia):</p>
        <ul>
            <li><b>Total de Necessidade:</b> {fmt_br(tot_necessidade)} unidades</li>
            <li><b>Total Sem Estoque:</b> {fmt_br(tot_sem_estoque)} unidades, o que representa <b>{pct_sem_estoque:.2f}%</b> da necessidade total</li>
            <li><b>Total Cancelado:</b> {fmt_br(tot_cancelado)} unidades, equivalente a <b>{pct_cancelado:.2f}%</b> da necessidade</li>
            <li><b>Receita Total:</b> R$ {fmt_br(tot_receita, eh_moeda=True)}, com volume atendido de {fmt_br(tot_atendido)} unidades</li>
        </ul>
        <p>Em anexo, estou enviando a listagem completa dos artigos sem estoque, assim como os cancelamentos abertos por franqueados.</p>
        <p>Qualquer dúvida, fico à disposição.</p>
        <p>Atenciosamente,<br>
    </body>
</html>
"""
msg.set_content(corpo_html, subtype='HTML', charset='utf-8')

# 3. ANEXAR E ENVIAR (Continua igual)
# ==============================================================================
with open(nome_arquivo_final, 'rb') as f: #"ler" o arquivo Excel antes de enviar. O 'rb' significa Read Binary (Leitura Binária)
    file_data = f.read() #guardamos o conteúdo do arquivo na memória do computador
    file_name = os.path.basename(nome_arquivo_final)  #Isso serve para que, quem receba o e-mail, veja apenas o nome do arquivo

msg.add_attachment(file_data, maintype='application', subtype='vnd.openxmlformats-officedocument.spreadsheetml.sheet', filename=file_name) #Esse nome gigante e assustador é apenas o código técnico oficial para "Arquivo Excel .xlsx". É isso que faz o Outlook ou Gmail do seu chefe mostrar o ícone verdinho do Excel quando o e-mail chega. Se você colocar o tipo errado, o computador dele não vai saber qual programa usar para abrir.

contexto = ssl.create_default_context() #Isso é segurança. O Gmail exige que a conexão seja criptografada.
try:
    with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=contexto) as smtp: #O SMTP (Simple Mail Transfer Protocol) é o protocolo padrão da internet para enviar e-mails.
        smtp.login(email_origem, senha_app) 
        smtp.send_message(msg) 
        print("E-mail com HTML enviado com sucesso!")
except Exception as e:
    print(f"Erro ao enviar: {e}")
